## Estilização da Saída do Markdown

In [32]:
%%HTML
<style>
  @import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono&display=swap');

  body {
    font-family: 'JetBrains Mono', -apple-system, BlinkMacSystemFont, 'Segoe WPC', 'Segoe UI', system-ui, 'Ubuntu', 'Droid Sans', sans-serif;
    line-height: 1.67;
  }
</style>

# Análise e Controle de Sistemas Lineares Incertos por meio de Desigualdades Matriciais Lineares: Exercício

## Descrição

Eeste notebook explora conceitos avançados de análise e controle de sistemas lineares incertos através da formulação e resolução de problemas de programação linear utilizando as bibliotecas CVX e Mosek em Python, com ênfase em Inequações Matriciais Lineares (LMIs).

### Data de Criação

2024-01-21

### Data de Modificação

2024-02-06

## Autor

Andevaldo da Encarnação Vitório  ([andevaldo.vitorio@icomp.ufam.edu.br](mailto:andevaldo.vitorio@icomp.ufam.edu.br))

## Orientador

Iury Valente de Bessa ([iury@ufam.edu.br](mailto:iury@ufam.edu.br))

## Importação das Bibliotecas Necessárias

In [22]:
import cvxpy as cp
import numpy as np
import math
from itertools import product

In [23]:
def show_matrix(name, matrix, decimal_places=2):
    """
    Apresenta uma matriz com a quantidade de casas decimais desejadas.

    Parâmetros:
    ---
    - matrix: numpy.ndarray, a matriz a ser apresentada.
    - casas_decimais: int, o número de casas decimais desejadas (padrão é 2).
    """
    pattern = "{:." + str(decimal_places) + "e}"

    def format_elem(elem):
        return pattern.format(elem)
    
    width = [max(map(len, map(format_elem, coluna))) for coluna in matrix.T]

    print(name, "=")

    nspaces = sum(width) + 2 * matrix.shape[1]

    print("    ┌" + " " * nspaces + "┐")
    for line in matrix:
        formatted_line = "  ".join(format_elem(e).rjust(largura) for e, largura in zip(line, width))
        print("    │ " + formatted_line + " │")
    print("    └" + " " * nspaces + "┘")
    print()


## Problema 1

### Descrição

Considere o seguinte problema de otimização convexo baseado em desigualdades matriciais lineares:

$$ \underset{P}{\min} \space Tr(P) $$
$$ P > 0, \space\space A^TP+PA+Q < 0 $$

Resolva com

$$
A = \begin{bmatrix}
0 & 1 \\
-2 & -3
\end{bmatrix}
, \space
Q = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$

In [24]:
# Problema 1 - Parte 1

# Definição da Variável P
P = cp.Variable((2,2), symmetric=True)

# Definição dos Parâmetros
A = cp.Parameter((2,2), value=[[0., -2.],[1, -3]])
Q = cp.Parameter((2,2), value=np.eye(2))

# Apresentação dos parâmetros
show_matrix('A', A.value)
show_matrix('Q', Q.value)

# Definição do problema de otimização convexa: objetivo e restrições
obj = cp.Minimize(cp.trace(P))
constraints = [P >> 0, A.T @ P + P @ A + Q << 0]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')
print("Status:", prob.status)

# Apresentação do resultado obtido na resolução
show_matrix('P', P.value)

A =
    ┌                      ┐
    │  0.00e+00   1.00e+00 │
    │ -2.00e+00  -3.00e+00 │
    └                      ┘

Q =
    ┌                    ┐
    │ 1.00e+00  0.00e+00 │
    │ 0.00e+00  1.00e+00 │
    └                    ┘


Valor ótimo: 1.4999999999962756 

Status: optimal
P =
    ┌                    ┐
    │ 1.25e+00  2.50e-01 │
    │ 2.50e-01  2.50e-01 │
    └                    ┘



Resolva agora com o seguinte problema de otimização alternativo:

$$ \underset{P,\space \rho}{\min}\space \rho $$
$$ P > 0, \space\space A^TP + PA + Q < 0, \space\space \rho \geq Tr(P) $$

In [25]:
# Problema 1 - Parte 2

# Definição da Variáveis P e rho
P = cp.Variable((2,2), symmetric=True)
ρ = cp.Variable()

# Definição dos Parâmetros
A = cp.Parameter((2,2), value=[[0., -2.],[1., -3.]])
Q = cp.Parameter((2,2), value=np.eye(2))

# Apresentação dos parâmetros
show_matrix('A', A.value)
show_matrix('Q', Q.value)

# Definição do problema de otimização convexa: objetivo e restrições
obj = cp.Minimize(ρ)
constraints = [P >> 0, A.T @ P + P @ A + Q << 0, ρ >= cp.trace(P)]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação do resultado obtido na resolução
show_matrix('P', P.value)
print("ρ = ", ρ.value)

A =
    ┌                      ┐
    │  0.00e+00   1.00e+00 │
    │ -2.00e+00  -3.00e+00 │
    └                      ┘

Q =
    ┌                    ┐
    │ 1.00e+00  0.00e+00 │
    │ 0.00e+00  1.00e+00 │
    └                    ┘


Valor ótimo: 1.4999999999962756 

P =
    ┌                    ┐
    │ 1.25e+00  2.50e-01 │
    │ 2.50e-01  2.50e-01 │
    └                    ┘

ρ =  1.4999999999962756


## Problema 2

### Descrição

Para ilustrar o cômputo da norma $\mathscr{H}_2 = \lVert H(s) \rVert_2$ usando LMIs, considere o sistema massa-mola apresentado em (Iwasaki, 1996), ilustrado na figura abaixo:

<br>
<div style="text-align:center;">
  <picture>
    <source media="(prefers-color-scheme: light)" srcset="assets/imgs/sistema_massa_mola_light.png">
    <source media="(prefers-color-scheme: dark)" srcset="assets/imgs/sistema_massa_mola_dark.png">
  
  <img src="assets/imgs/sistema_massa_mola_dark.png" alt="Sistema Massa Mola" width="60%"   title="Sistema Massa Mola">
  </picture>
</div>
<br>

A mesma função de transferência de (Iwasaki, 1996) é considerada, isto é, da força de entrada $d$ aplicada à massa $m_1$ para o sinal de erro $e = x_2$ (posição da massa $m_2$). O espaço de estado deste sistema é:

<br>

$$
\begin{bmatrix}
\dot{x_1} \\ \dot{x_2} \\ \ddot{x_1} \\ \ddot{x_2}
\end{bmatrix}

= 

\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\displaystyle-\frac{k_1 + k_2}{m_1} & \displaystyle\frac{k_2}{m_1} &\displaystyle-\frac{c_0}{m_1} & 0 \\
\displaystyle\frac{k_2}{m_2} &\displaystyle-\frac{k_2}{m_2} &0 &\displaystyle-\frac{c_0}{m_2}
\end{bmatrix}

\begin{bmatrix}
x_1 \\ x_2 \\ \dot{x_1} \\ \dot{x_2}
\end{bmatrix}

+

\begin{bmatrix}
0 \\
0 \\
\displaystyle \frac{1}{m_1} \\
0 \\
\end{bmatrix}

d
$$

<br>

$$
Y =
\begin{bmatrix}
0 & 1 & 0 & 0
\end{bmatrix}

\begin{bmatrix}
x_1 \\ x_2 \\ \dot{x_1} \\ \dot{x_2}
\end{bmatrix}
$$

<br>

Portanto, as matrizes $A$, $B$ e $C$ são dadas por

<br>

$$
A = \begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\displaystyle-\frac{k_1 + k_2}{m_1} & \displaystyle\frac{k_2}{m_1} &\displaystyle-\frac{c_0}{m_1} & 0 \\
\displaystyle\frac{k_2}{m_2} &\displaystyle-\frac{k_2}{m_2} &0 &\displaystyle-\frac{c_0}{m_2}
\end{bmatrix}
, \space
B =
\begin{bmatrix}
0 \\
0 \\
\displaystyle \frac{1}{m_1} \\
0 \\
\end{bmatrix}
, \space
C =
\begin{bmatrix}
0 & 1 & 0 & 0
\end{bmatrix}
, \space
D = 0
$$

<br>

Os valores dos parâmetros são: massas $m_1 = 1$ e $m_2 = 0.5$; elasticidade das molas $k_1 = k_2 = 1$ e as forças de atrito $f_1$ e $f_2$ dadas por $f_i = −c_0 \cdot \dot{x_i}, \space (i = 1, 2)$, sendo que
$c_0 = 2$ é o coeficiente de atrito viscoso.

Determine a norma $\mathscr{H} _2$ deste sistema usando o seguinte problema de otimização:

$$ \underset{\rho, \space W = W^T > 0}{\min} \space\space \rho $$

sujeito a

$$
rho \geq Tr(CWC^T), \space
AW + WA^T + BB^T < 0
$$

Na solução ótima $\rho = \rho^\star$, tem se

$$ \lVert H(s) \rVert ^ 2 _ 2 = \rho^\star $$

onde, $H(s)$ é a matriz de transferência do sistema, dada por:

$$ H(s) = C(sI- A)^{-1}B +D  $$

In [33]:
# Problema 2

# Definição dos Parâmetros
m1 = 1.
m2 = 0.5
k1 = 1.
k2 = 1.
c0 = 2.

A = cp.Parameter((4, 4), value=[[0., 0., - ((k1 + k2) / m1), (k2 / m2)],
                               [0., 0., (k2 / m1), - (k2 / m2)],
                               [1., 0., - (c0 / m1), 0],
                               [0., 1., 0., -(c0 / m2)]])
B = cp.Parameter((4, 1), value=[[0., 0., 1. / m1, 0.]])
C = cp.Parameter((1, 4), value=[[0], [1], [0], [0]])

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B', B.value)
show_matrix('C', C.value)

# Definição das Variáveis
ρ = cp.Variable()
W = cp.Variable((4, 4), symmetric=True)

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(ρ)
constraints = [W >> 0]
constraints += [ρ >= cp.trace(C @ W @ C.T)]
constraints += [A @ W + W @ A.T + B @ B.T << 0]
prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação dos Resultados
show_matrix('W', W.value)
print("ρ = ", ρ.value, '\n')
print("Norma H2 = ", math.sqrt(ρ.value), '\n')

Parâmetros:

A =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -2.00e+00   1.00e+00  -2.00e+00   0.00e+00 │
    │  2.00e+00  -2.00e+00   0.00e+00  -4.00e+00 │
    └                                            ┘

B =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    │ 0.00e+00 │
    └          ┘

C =
    ┌                                        ┐
    │ 0.00e+00  1.00e+00  0.00e+00  0.00e+00 │
    └                                        ┘


Valor ótimo: 0.0925925924451939 

W =
    ┌                                            ┐
    │  1.57e-01   8.33e-02  -5.53e-09   3.70e-02 │
    │  8.33e-02   9.26e-02  -3.70e-02  -2.52e-10 │
    │ -5.53e-09  -3.70e-02   2.31e-01   1.00e-09 │
    │  3.70e-02  -2.52e-10   1.00e-09   1.85e-02 │
    └                                            ┘

ρ =  0.0925925924451939 

Norma H2 =  0.30429030948289154 



## Problema 3

### Descrição

Determine a norma $\mathscr{H}_{\infty}$ do sistema massa-mola usando o seguinte problema de otimização:

$$ \underset{P = P^T > 0}{\min} \space \space \mu $$

sujeito a

$$
\begin{bmatrix}
A^TP + PA + C^TC & PB + C^TD\\
B^TP + D^TC & D^TD - \mu
\end{bmatrix} 
< 0
$$

No ótimo $\mu = \mu^{\star}$, a norma $\mathscr{H}_{\infty}$ é dada por

$$ \lVert H(s) \rVert _ \infty = \sqrt{\mu ^ \star}$$

In [27]:
# Problema 3

# Definição dos Parâmetros
m1 = 1.
m2 = 0.5
k1 = 1.
k2 = 1.
c0 = 2.

A = cp.Parameter((4, 4), value=[[0., 0., - ((k1 + k2) / m1), (k2 / m2)],
                                [0., 0., (k2 / m1), - (k2 / m2)],
                                [1., 0., - (c0 / m1), 0],
                                [0., 1., 0., -(c0 / m2)]])
B = cp.Parameter((4, 1), value=[[0., 0., 1. / m1, 0.]])
C = cp.Parameter((1, 4), value=[[0], [1], [0], [0]])
D = cp.Parameter((1, 1), value=[[0]])
I = cp.Parameter((1, 1), value=np.identity(1))

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B', B.value)
show_matrix('C', C.value)
show_matrix('D', D.value)

# Definição das Variáveis
P = cp.Variable((4, 4), symmetric=True)
µ = cp.Variable()
M = cp.bmat([[A.T @ P + P @ A + C.T @ C, P @ B + C.T @ D], 
            [B.T @ P + D.T @ C, D.T @ D - µ * I]])


# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(µ)
constraints = [P >> 0, M << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("\nValor ótimo:", prob.value, '\n')

# Apresentação dos Resultados
show_matrix('P', P.value)
print("µ = ", µ.value, '\n')
print("Norma Hinf = ", math.sqrt(µ.value), '\n')

Parâmetros:

A =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -2.00e+00   1.00e+00  -2.00e+00   0.00e+00 │
    │  2.00e+00  -2.00e+00   0.00e+00  -4.00e+00 │
    └                                            ┘

B =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    │ 0.00e+00 │
    └          ┘

C =
    ┌                                        ┐
    │ 0.00e+00  1.00e+00  0.00e+00  0.00e+00 │
    └                                        ┘

D =
    ┌          ┐
    │ 0.00e+00 │
    └          ┘


Valor ótimo: 0.9999999978766486 

P =
    ┌                                          ┐
    │  2.02e+00  -2.37e-02  4.95e-01  2.00e-01 │
    │ -2.37e-02   4.02e+00  5.05e-01  8.00e-01 │
    │  4.95e-01   5.05e-01  9.40e-01  1.63e-01 │
    │  2.00e-01   8.00e-01  1.63e-01  6.39e-01 │
    └                                          ┘

µ =  0.9999999978766486 

Norma H

## Exercício 4

Considere um modelo de satélite (Gahinet et al., 1995) que consiste em dois corpos rı́gidos (corpo principal e de instrumentação) conectados por um ligação flexı́vel modelada como uma mola com constante de torque k e amortecimento viscoso $f$. A Figura 2 apresenta uma ilustração do satélite, sendo $\theta_1$ e $\theta_2$ os ângulos de guinada para o corpo principal e para o módulo de instrumentação, respectivamente.

<br>

$$
\begin{bmatrix}
\dot{\theta_1} \\ \dot{\theta_2} \\ \ddot{\theta_1} \\ \ddot{\theta_2}
\end{bmatrix}

= 

\begin{bmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
-k & k & -f & f \\
k & -k & f & -f
\end{bmatrix}

\begin{bmatrix}
\theta_1 \\ \theta_2 \\ \dot{\theta_1} \\ \dot{\theta_2}
\end{bmatrix}

+

\begin{bmatrix}
0 \\
0 \\
0 \\
1 \\
\end{bmatrix}

w

+

\begin{bmatrix}
0 \\
0 \\
0 \\
1 \\
\end{bmatrix}

T

$$

<br>

$$
y =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix}

\begin{bmatrix}
\theta_1 \\ \theta_2 \\ \dot{\theta_1} \\ \dot{\theta_2}
\end{bmatrix}

+ 

\begin{bmatrix}
0 \\
0 \\
1 \\
\end{bmatrix}

T
$$

<br>

sendo T o torque de controle e w uma perturbação sobre o corpo principal.

$$ \underset{\rho, \space X = X^T, \space Z, \space W = W^T > 0}{\min} \space \space \rho $$

sujeito a

<br>

$$
\begin{bmatrix}
X & CW + DZ  \\
WC^T + Z^TD^T & W 
\end{bmatrix} 
> 0

, \space

\rho \geq Tr(X)

, \space

\begin{bmatrix}
AW + B_2Z + WA^T + Z^TB^T_2 & B_1 \\
B_1^T & -I 
\end{bmatrix} 
< 0
$$

<br>

com $K = ZW^{-1}$ e $\lVert H(s) \rVert ^2_2 = \rho^\star$.

In [28]:
# Problema 4

# Definição dos Parâmetros
k = 0.2
f = 0.0039
J1 = 1.
J2 = 1.

A = cp.Parameter((4, 4), value=[[0., 0., -k, k],
                                [0., 0., k, -k],
                                [1., 0., -f, f],
                                [0., 1., f, -f]])


B1 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
B2 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
C = cp.Parameter((3, 4), value=[[1, 0, 0],
                                [0, 1, 0],
                                [0, 0, 0],
                                [0, 0, 0]])

D = cp.Parameter((3, 1), value=[[0, 0, 1]])

I = cp.Parameter((1, 1), value=np.identity(1))

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B1', B1.value)
show_matrix('B2', B2.value)
show_matrix('C', C.value)
show_matrix('D', D.value)

# Definição das Variáveis
ρ = cp.Variable()
X = cp.Variable((3, 3), symmetric=True)
Z = cp.Variable((1, 4))
W = cp.Variable((4, 4), symmetric=True)

zeros = np.zeros((1, 4))

M11 = X
M12 = C @ W + D @ Z
M21 = W @ C.T + Z.T @ D.T
M22 = W

M = cp.bmat([[M11, M12],
             [M21, M22]])

N11 = A @ W + B2 @ Z + W @ A.T + Z.T @ B2.T
N12 = B1
N21 = B1.T
N22 = -I

N = cp.bmat([[N11, N12],
             [N21, N22]])


# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(ρ)
constraints = [W >> 0]
constraints += [M >> 0]
constraints += [ρ >= cp.trace(X)]
constraints += [N << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("Valor ótimo:", prob.value, '\n')

K = np.dot(Z.value, np.linalg.inv(W.value))

# Apresentação dos Resultados
print("ρ = ", ρ.value, '\n')
show_matrix("X", X.value)
show_matrix("Z", Z.value)
show_matrix("W", W.value)
show_matrix("K", K)

print("Norma H2 = ", math.sqrt(ρ.value), '\n')

Parâmetros:

A =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -2.00e-01   2.00e-01  -3.90e-03   3.90e-03 │
    │  2.00e-01  -2.00e-01   3.90e-03  -3.90e-03 │
    └                                            ┘

B1 =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

B2 =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

C =
    ┌                                        ┐
    │ 1.00e+00  0.00e+00  0.00e+00  0.00e+00 │
    │ 0.00e+00  1.00e+00  0.00e+00  0.00e+00 │
    │ 0.00e+00  0.00e+00  0.00e+00  0.00e+00 │
    └                                        ┘

D =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

Valor ótimo: 1.659660370311429 

ρ =  1.659660370311429 

X =
    ┌                                 ┐
    │  1.66e-01  -1.17e-02  

## Exercício 5

Determine um controlador por realimentação de estados $\mathscr{H}_\infty$ para o modelo do satélite usando o problema de otimização:

$$ \underset{Z, \space W = W^T > 0}{\min} \space \space \mu $$

sujeito a

<br>

$$
\begin{bmatrix}
AW+WA^T+B_2Z+Z^TB_2^T & WC^T+Z^TD^T & B_1 \\
CW+DZ & -I & 0 \\
B_1^T & 0 & -\mu I 
\end{bmatrix} 
< 0
$$

<br>

Na solução ótima, $K = ZW^{-1}$ e $\lVert H(s) \rVert _\infty = \sqrt{\mu^\star}$.

In [29]:
# Problema 5

# Definição dos Parâmetros
k = 0.2
f = 0.0039
J1 = 1.
J2 = 1.

A = cp.Parameter((4, 4), value=[[0., 0., -k, k],
                                [0., 0., k, -k],
                                [1., 0., -f, f],
                                [0., 1., f, -f]])


B1 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
B2 = cp.Parameter((4, 1), value=[[0., 0., 0., 1. / J1]])
C = cp.Parameter((3, 4), value=[[1, 0, 0],
                                [0, 1, 0],
                                [0, 0, 0],
                                [0, 0, 0]])

D = cp.Parameter((3, 1), value=[[0, 0, 1]])

I1 = cp.Parameter((1, 1), value=np.identity(1))
I3 = cp.Parameter((3, 3), value=np.identity(3))

print('Parâmetros:\n')
show_matrix('A', A.value)
show_matrix('B1', B1.value)
show_matrix('B2', B2.value)
show_matrix('C', C.value)
show_matrix('D', D.value)

zeros = np.zeros((1, 4))

# Definição das Variáveis
µ = cp.Variable()
Z = cp.Variable((1, 4))
W = cp.Variable((4, 4), symmetric=True)

M11 = A @ W + W @ A.T + B2 @ Z + Z.T @ B2.T
M12 = W @ C.T + Z.T @ D.T
M13 = B1

M21 = C @ W + D @ Z
M22 = -I3
M23 = np.zeros((3, 1))

M31 = B1.T
M32 = np.zeros((1, 3))
M33 = -µ * I1

M = cp.bmat([[M11, M12, M13],
             [M21, M22, M23],
             [M31, M32, M33]])

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(µ)
constraints = [M << 0]
constraints += [W >> 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("Valor ótimo:", prob.value, '\n')

K = np.dot(Z.value, np.linalg.inv(W.value))

# Apresentação dos Resultados
print("µ = ", µ.value, '\n')
show_matrix("Z", Z.value)
show_matrix("K", K)

print("Norma H∞ = ", math.sqrt(µ.value), '\n')

Parâmetros:

A =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -2.00e-01   2.00e-01  -3.90e-03   3.90e-03 │
    │  2.00e-01  -2.00e-01   3.90e-03  -3.90e-03 │
    └                                            ┘

B1 =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

B2 =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

C =
    ┌                                        ┐
    │ 1.00e+00  0.00e+00  0.00e+00  0.00e+00 │
    │ 0.00e+00  1.00e+00  0.00e+00  0.00e+00 │
    │ 0.00e+00  0.00e+00  0.00e+00  0.00e+00 │
    └                                        ┘

D =
    ┌          ┐
    │ 0.00e+00 │
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

Valor ótimo: 0.9999999791446339 

µ =  0.9999999791446339 

Z =
    ┌                                          ┐
    │ -7.63e-05  

## Exercício 6

Considere novamente o modelo do satélite. A constante da mola e o amortecimento viscoso não mais são assumidos como precisamente conhecidos, mas pertencentes às faixas

$$ 0.09 \leq k \leq 0.4, \space 0.0038 \leq f \leq 0.04 $$

Armazene os vértices do sistema em uma estrutura celular.

Calcule um controlador estabilizante com um custo garantido $\mathscr{H} _ 2$ usando oseguinte problema de otimização.

$$ \underset{rho, \space X = X^T, \space Z, \space W = W^T > 0}{\min} \space \space \rho $$

sujeito a

<br>

$$

\begin{bmatrix}
X & B^T_{1i} \\
B_{1i} & W \\
\end{bmatrix} 
> 0

, \space

\begin{bmatrix}
A_iW+WA_i^T+B_{2i}Z+Z^TB_{2i}^T & WC_i^T+Z^TD_i^T\\rh
C_iW+D_iZ & -I \\
\end{bmatrix} 
< 0

, \space 

i = 1, ..., N

$$

$$ \rho \geq Tr(X) $$

<br>

Na solução ótima $\rho = \rho^\star$ é um custo garantido $\mathscr{H}_2$ (isto é, $\lVert H(s) \rVert \leq \rho$) para o sistema em malha fechada realimentando com o ganho robusto $K = ZW^{-1}$.


In [30]:
# Problema 6

# Definição dos Parâmetros
k_min = 0.09
k_max = 0.4
f_min = 0.0038
f_max = 0.04

k_int = [k_min, k_max]
f_int = [f_min, f_max]

A = []
for k in k_int:
  for f in f_int:
    A += [cp.Parameter((4, 4), value=[[0., 0., -k, k],
                                      [0., 0., k, -k],
                                      [1., 0., -f, f],
                                      [0., 1., f, -f]])]


B1 = cp.Parameter((4, 1), value=[[0., 0., 0., 1.]])
B2 = cp.Parameter((4, 1), value=[[0., 0., 0., 1.]])
C = cp.Parameter((3, 4), value=[[1, 0, 0],
                                [0, 1, 0],
                                [0, 0, 0],
                                [0, 0, 0]])

D = cp.Parameter((3, 1), value=[[0, 0, 1]])

I1 = cp.Parameter((1, 1), value=np.identity(1))
I3 = cp.Parameter((3, 3), value=np.identity(3))

print('Parâmetros:\n')

for i in range(len(A)):
  show_matrix(f'A[{i}]', A[i].value)
show_matrix('B1', B1.value)
show_matrix('B2', B2.value)
show_matrix('C', C.value)
show_matrix('D', D.value)

zeros = np.zeros((1, 4))

# Definição das Variáveis
ρ = cp.Variable()
Z = cp.Variable((1, 4))
X = cp.Variable((1, 1), symmetric=True)
W = cp.Variable((4, 4), symmetric=True)

M11 = X
M12 = B1.T
M21 = B1
M22 = W

M = cp.bmat([[M11, M12],
             [M21, M22]])

N = []

for Ai in A:
  N11 = Ai @ W + W @ Ai.T + B2 @ Z + Z.T @ B2.T
  N12 = W @ C.T + Z.T @ D.T

  N21 = C @ W + D @ Z
  N22 = -I3

  N += [cp.bmat([[N11, N12],
                 [N21, N22]])]


# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(ρ)
constraints = [M >> 0]
constraints += [ρ >= cp.trace(X)]
constraints += [W >> 0]

for Ni in N:
  constraints += [Ni << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)
print("Valor ótimo:", prob.value, '\n')

K = np.dot(Z.value, np.linalg.inv(W.value))

# Apresentação dos Resultados
print("ρ = ", ρ.value, '\n')
show_matrix("Z", Z.value)
show_matrix("K", K)

print("Norma H2 = ", math.sqrt(ρ.value), '\n')

Parâmetros:

A[0] =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -9.00e-02   9.00e-02  -3.80e-03   3.80e-03 │
    │  9.00e-02  -9.00e-02   3.80e-03  -3.80e-03 │
    └                                            ┘

A[1] =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -9.00e-02   9.00e-02  -4.00e-02   4.00e-02 │
    │  9.00e-02  -9.00e-02   4.00e-02  -4.00e-02 │
    └                                            ┘

A[2] =
    ┌                                            ┐
    │  0.00e+00   0.00e+00   1.00e+00   0.00e+00 │
    │  0.00e+00   0.00e+00   0.00e+00   1.00e+00 │
    │ -4.00e-01   4.00e-01  -3.80e-03   3.80e-03 │
    │  4.00e-01  -4.00e-01   3.80e-03  -3.80e-03 │
    └                                            ┘

A[3] =
    ┌                                 

## Exercício 7

Considere o sistema dinâmico incerto descrito descrito por

$$ \dot{x} = A(\alpha)x, \space \alpha \in \Delta $$

cujos vértices $A_i$ são dados por

$$ 
A_i =  M_0 + \sum_{i=1}^{3} \left(\rho_iM_i \right) , \space
onde \space \rho_i \in \{ 0, \space 1 \}
$$

com

$$

M_0 =
\begin{bmatrix}
-2.4 & -0.6 & -1.7 & 3.1 \\
0.7 & -2.1 & -2.6 & -3.6 \\
0.5 & 2.4 & -5.0 & -1.6 \\
-0.6 & 2.9 & -2.0 & -0.6 \\
\end{bmatrix}
, \space
M_1 =
\begin{bmatrix}
1.1 & -0.6 & -0.3 & -0.1 \\
-0.8 & -0.2 & -1.1 & 2.8 \\
-1.9 & 0.8 & -1.1 & 2.0 \\
-2.4 & -3.1 & -3.7 & -0.1 \\
\end{bmatrix}


$$

$$

M_2 =
\begin{bmatrix}
-0.9 & -3.4 & 1.7 & 1.5 \\
-3.4 & -1.4 & 1.3 & 1.4 \\
1.1 & 2.0 & -1.5 & -3.4 \\
-0.4 & 0.5 & 2.3 & 1.5 \\
\end{bmatrix}
, \space
M_3 =
\begin{bmatrix}
-1.0 & -1.4 & -0.7 & -0.7 \\
2.1 & 0.6 & -0.1 & -2.1 \\
0.4 & -1.4 & 1.3 & 0.7 \\
1.5 & 0.9 & 0.4 & -0.5 \\
\end{bmatrix}

$$

<br>

**Problema de factibilidade 1:** Se existirem matrizes simétricas definidas
positivas $P_i \in \real^{n \times n}, \space i = 1, ..., N$ e matrizes $X_1 \in \real^{n \times n}$ e $X_2 \in \real^{n \times n}$ tais que

$$
\begin{bmatrix}
X_1A_i+A_i^TX_1^T & P_i-X_1+A_i^TX_2^T \\
P_i+X_2A_i-X_1^T & -X_2-X_2^T
\end{bmatrix}
< 0
, \space
i = 1, ..., N
$$

In [31]:
# Problema 7

# Definição dos Parâmetros
M = 4 * [None]
M[0] = np.array([[-0.9, -3.4, 1.7, 1.5],
                 [-3.4, -1.4, 1.3, 1.4],
                 [1.1, 2.0, -1.5, -3.4],
                 [-0.4, 0.5, 2.3, 1.5]])
M[1] = np.array([[1.1, -0.6, -0.3, -0.1],
                 [-0.8, -0.2, -1.1, 2.8],
                 [-1.9, 0.8, -1.1, 2.0],
                 [-2.4, -3.1, -3.7, -0.1]])
M[2] = np.array([[-0.9, -3.4, 1.7, 1.5],
                 [-3.4, -1.4, 1.3, 1.4],
                 [1.1, 2.0, -1.5, -3.4],
                 [-0.4, 0.5, 2.3, 1.5]])
M[3] = np.array([[-1.0, -1.4, -0.7, -0.7],
                 [2.1, 0.6, -0.1, -2.1],
                 [0.4, -1.4, 1.3, 0.7],
                 [1.5, 0.9, 0.4, -0.5]])

rho_combinations = product(range(2), repeat=3)
A = []
i = 0
for rhos in rho_combinations:
  Ai = M[0] + sum(rho * M_i for rho, M_i in zip(rhos, M[1:]))
  A.append(cp.Parameter((4, 4), value=Ai, name=f'A{i}'))
  i += 1

print('Parâmetros:\n')

for i in range(len(A)):
  show_matrix(f'A[{i}]', A[i].value)

# Definição das Variáveis
X1 = cp.Variable((4, 4), name='X1')
X2 = cp.Variable((4, 4), name='X2')
P = [cp.Variable((4, 4), symmetric=True, name=f'P{i}') for i in range(len(A))]

M = []

for i in range(len(A)):
  M11 = X1 @ A[i] + A[i].T @ X1.T
  M12 = P[i] - X1 + A[i].T @ X2.T
  M21 = P[i] + X2 @ A[i] - X1.T
  M22 = -X2 - X2.T

  M.append(cp.bmat([[M11, M12],
                    [M21, M22]]))

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(0)
constraints = []
for Pi in P:
  constraints += [Pi >> 0]
for Mi in M:
  constraints += [Mi << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)

if prob.status not in ["infeasible", "unbounded"]:
  # Otherwise, prob.value is inf or -inf, respectively.
  print("Valor ótimo: %s\n" % prob.value)
  for variable in prob.variables():
    show_matrix(variable.name(), variable.value)
else:
  print('O problema não é factível')

Parâmetros:

A[0] =
    ┌                                            ┐
    │ -9.00e-01  -3.40e+00   1.70e+00   1.50e+00 │
    │ -3.40e+00  -1.40e+00   1.30e+00   1.40e+00 │
    │  1.10e+00   2.00e+00  -1.50e+00  -3.40e+00 │
    │ -4.00e-01   5.00e-01   2.30e+00   1.50e+00 │
    └                                            ┘

A[1] =
    ┌                                            ┐
    │ -1.90e+00  -4.80e+00   1.00e+00   8.00e-01 │
    │ -1.30e+00  -8.00e-01   1.20e+00  -7.00e-01 │
    │  1.50e+00   6.00e-01  -2.00e-01  -2.70e+00 │
    │  1.10e+00   1.40e+00   2.70e+00   1.00e+00 │
    └                                            ┘

A[2] =
    ┌                                            ┐
    │ -1.80e+00  -6.80e+00   3.40e+00   3.00e+00 │
    │ -6.80e+00  -2.80e+00   2.60e+00   2.80e+00 │
    │  2.20e+00   4.00e+00  -3.00e+00  -6.80e+00 │
    │ -8.00e-01   1.00e+00   4.60e+00   3.00e+00 │
    └                                            ┘

A[3] =
    ┌                                 